POS Tagging with HMM and Sentence Generation 

The training dataset is a subset of the Brown corpus, where each file contains sentences in the form of tokenized words followed by POS tags. Each line contains one sentence. Training dataset can be downloaded from here: https://bit.ly/2kJI0yc The test dataset (which is another subset of the Brown corpus, containing tokenized words but no tags) can be downloaded from here: https://bit.ly/2lMybzP Information regarding the categories of the dataset can be found at: https://bit.ly/2mhF6RT.

Your task is to implement a part-of-speech tagger using a bi-gram HMM. Given an observation
sequence of n words wn1, choose the most probable sequence of POS tags tn1. For the questionsbelow, please submit both code and output.

[Note: During training, for a word to be counted as unknown, the frequency of the word in
training set should not exceed a threshold (e.g. 5). You can pick a threshold based on your algorithm design. Also, you can implement smoothing technique based on your own choice, e.g.
add-α.]

In [1]:
import glob
import re
import math
import random
import pandas as pd
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
def load_data(input_directory):
    word_tokens = []
    tag_tokens = []
    word_tag_tokens = []
    transition_tag_tokens = []
    sentence_count = 0
    for file_name in glob.glob(input_directory + "/*"):
        print("Prepocessing: {}".format(file_name))
        file_pointer = open(file_name, "r")
        
        for line in file_pointer:

            # Remove duplicate spaces
            file_line_content = re.sub(' +', ' ', line)

            # Remove new line characters
            file_line_content = line.replace("\n", " ")
            
            # Strip of begin and end spaces
            file_line_content = file_line_content.strip()
            
            # If line is not empty
            if file_line_content != "":
                sentence_count = sentence_count + 1
                line_content_list = file_line_content.split(" ")
                
                # Append start tag
                transition_tag_tokens.append('START')
#                 print(line_content_list)
                for i in line_content_list:
                    word_tag_tokens.append(i)
                    split_tokens = i.split('/')
                    word = split_tokens[0]
                    tag = split_tokens[-1]
                    word_tokens.append(word)
                    tag_tokens.append(tag)
                    transition_tag_tokens.append(tag)
    
                # Append end tag
                transition_tag_tokens.append('END')
        
        file_pointer.close()
        
    return sentence_count, word_tokens, tag_tokens, word_tag_tokens, transition_tag_tokens

In [3]:
def replace_low_count_words(word_tokens, cut_off_count, word_tag_tokens):
    word_tokens_with_count = Counter(word_tokens)
    candidate_words = {}
    for word in word_tokens_with_count:
        if word_tokens_with_count[word] <= cut_off_count:
            candidate_words[word] = 1
    
    ## Remove all the words <= cut_off_count
    for i in range(len(word_tokens)):
        if word_tokens[i] in candidate_words:
            word_tokens[i] = 'UNK'
            split_tokens = word_tag_tokens[i].split('/')
            word = split_tokens[0]
            tag = split_tokens[-1]
            word_tag_tokens[i] = 'UNK' + '/' + tag

    return word_tokens, word_tag_tokens

In [4]:
print("##### Loading train date")
# train_sentence_count, train_word_tokens, train_tag_tokens, train_word_tag_tokens, \
#     train_transition_tag_tokens = load_data('../input/custom_train')
    
train_sentence_count, train_word_tokens, train_tag_tokens, train_word_tag_tokens, \
    train_transition_tag_tokens = load_data('../input/brown_train')
    
print(" Number of Sentences: {}, Word list count: {}, Tag list count: {}, Transition Tag list count: {}"\
      " Word Tag List"
       .format(train_sentence_count, len(train_word_tokens), len(train_tag_tokens), len(train_word_tag_tokens), len(train_transition_tag_tokens))
      )

print("### Replace word tokens <= 5 with 'UNK")
train_word_tokens, train_word_tag_tokens = replace_low_count_words(train_word_tokens, 1, train_word_tag_tokens)
print("Number of tokens after replacement: Word - {}, Word_Tag - {}".format(len(train_word_tokens), len(train_word_tag_tokens)))


##### Loading train date
Prepocessing: ../input/brown_train/cd05
Prepocessing: ../input/brown_train/cf37
Prepocessing: ../input/brown_train/cf08
Prepocessing: ../input/brown_train/cl06
Prepocessing: ../input/brown_train/cj68
Prepocessing: ../input/brown_train/cf30
Prepocessing: ../input/brown_train/cd02
Prepocessing: ../input/brown_train/cl01
Prepocessing: ../input/brown_train/cj57
Prepocessing: ../input/brown_train/cf06
Prepocessing: ../input/brown_train/cl08
Prepocessing: ../input/brown_train/cj61
Prepocessing: ../input/brown_train/cf39
Prepocessing: ../input/brown_train/cn05
Prepocessing: ../input/brown_train/cj59
Prepocessing: ../input/brown_train/cf01
Prepocessing: ../input/brown_train/cn02
Prepocessing: ../input/brown_train/cj66
Prepocessing: ../input/brown_train/cj32
Prepocessing: ../input/brown_train/ch07
Prepocessing: ../input/brown_train/cj35
Prepocessing: ../input/brown_train/cb09
Prepocessing: ../input/brown_train/cj03
Prepocessing: ../input/brown_train/cj04
Prepocessing: .

Prepocessing: ../input/brown_train/cd06
Prepocessing: ../input/brown_train/cj53
Prepocessing: ../input/brown_train/cl05
Prepocessing: ../input/brown_train/cf02
Prepocessing: ../input/brown_train/cn01
Prepocessing: ../input/brown_train/cj65
Prepocessing: ../input/brown_train/cf05
Prepocessing: ../input/brown_train/cj62
Prepocessing: ../input/brown_train/cn06
Prepocessing: ../input/brown_train/cd08
Prepocessing: ../input/brown_train/cj01
Prepocessing: ../input/brown_train/cb02
Prepocessing: ../input/brown_train/cj06
Prepocessing: ../input/brown_train/cb05
Prepocessing: ../input/brown_train/cj39
Prepocessing: ../input/brown_train/cj30
Prepocessing: ../input/brown_train/ch02
Prepocessing: ../input/brown_train/ch05
Prepocessing: ../input/brown_train/cj37
Prepocessing: ../input/brown_train/cj08
Prepocessing: ../input/brown_train/cf04
Prepocessing: ../input/brown_train/cj63
Prepocessing: ../input/brown_train/cd09
Prepocessing: ../input/brown_train/cn07
Prepocessing: ../input/brown_train/cf03


Prepocessing: ../input/brown_train/cj71
Prepocessing: ../input/brown_train/cl18
Prepocessing: ../input/brown_train/cf16
Prepocessing: ../input/brown_train/cj76
Prepocessing: ../input/brown_train/cl20
Prepocessing: ../input/brown_train/cn12
Prepocessing: ../input/brown_train/cf11
Prepocessing: ../input/brown_train/cj49
Prepocessing: ../input/brown_train/cf45
Prepocessing: ../input/brown_train/cb21
Prepocessing: ../input/brown_train/ch10
Prepocessing: ../input/brown_train/cj22
Prepocessing: ../input/brown_train/ch28
Prepocessing: ../input/brown_train/cb26
Prepocessing: ../input/brown_train/cf42
Prepocessing: ../input/brown_train/cj25
Prepocessing: ../input/brown_train/cb19
Prepocessing: ../input/brown_train/ch17
Prepocessing: ../input/brown_train/cb10
Prepocessing: ../input/brown_train/cj13
Prepocessing: ../input/brown_train/ch21
Prepocessing: ../input/brown_train/cb17
Prepocessing: ../input/brown_train/ch19
Prepocessing: ../input/brown_train/ch26
Prepocessing: ../input/brown_train/cj14


4.1
Obtain frequency counts from the collection of all the training files (counted together). You will need the following types of frequency counts: word-tag counts, tag un-igram counts, and tag bigram counts. Let’s denote these by C(wi, ti), C(ti) and C(ti−1, ti) respectively. Report these quantities in different output files.

In [5]:
def get_unigrams(token_list):
    tokens_with_count = Counter(token_list)
    return tokens_with_count

In [6]:
def get_ngrams(token_list, n):
    ngrams_zip = zip(*[token_list[i:] for i in range(n)])
    ngrams_list = [" ".join(element) for element in ngrams_zip]
    ngrams_keys_counts = Counter(ngrams_list)
    return ngrams_keys_counts

In [7]:
print("Unigragms for words")
train_word_unigrams = get_unigrams(train_word_tokens)
print(len(train_word_unigrams))
vocab_size = len(train_word_unigrams)
print("Vocab size {}".format(vocab_size))

print("Unigragms for word-tag")
print(len(train_word_tag_tokens))
train_word_tag_unigrams = get_unigrams(train_word_tag_tokens)
with open('../output/word_tag_unigrams.txt', 'w') as word_tag_unigrams_output_file:
    word_tag_unigrams_output_file.write(str(train_word_tag_unigrams))
print(len(train_word_tag_unigrams))

print("Unigragms for tags")
train_tag_unigrams = get_unigrams(train_tag_tokens)
print(len(train_tag_unigrams))

print("Unigrams for transition tags")
train_transition_tag_unigrams = get_unigrams(train_transition_tag_tokens)
with open('../output/tag_unigrams.txt', 'w') as transition_tag_unigrams_output_file:
    transition_tag_unigrams_output_file.write(str(train_transition_tag_unigrams))
print(len(train_transition_tag_unigrams))

print("Bigrams for transition tags")
train_transition_tag_bigrams = get_ngrams(train_transition_tag_tokens, 2)
with open('../output/tag_bigrams.txt', 'w') as transition_tag_bigrams_output_file:
    transition_tag_bigrams_output_file.write(str(train_transition_tag_bigrams))
print(len(train_transition_tag_bigrams))

Unigragms for words
30036
Vocab size 30036
Unigragms for word-tag
1126281
41029
Unigragms for tags
470
Unigrams for transition tags
472
Bigrams for transition tags
8255


4.2  
A transition probability is the probability of a tag given its previous tag. Calculate transition
probabilities of the training set using the following equation:  
P(ti−1, ti) = C(ti−1, ti)/C(ti−1)

In [8]:
def get_transition_probability(tag_unigrams, tag_bigrams, lambda_value, vocab_size):
    transition_probability = {}
    for i in tag_bigrams:
        previous_tag = i.split(" ")[0]
        transition_probability[i] = (tag_bigrams[i] + lambda_value) / (tag_unigrams[previous_tag] + (lambda_value * vocab_size))
    return transition_probability

In [9]:
print("Get transition probability")
train_transition_probability = get_transition_probability(train_transition_tag_unigrams, train_transition_tag_bigrams, 0.1, vocab_size)
# print(train_transition_probability)

Get transition probability


4.3  
An emission probability is the probability of a given word being associated with a given tag. Calculate emission probabilities of the training set using the following equation:  
P(wi, ti) = C(wi, ti)/C(ti)

In [10]:
def get_emission_probability(word_tag_unigrams, tag_unigrams, lambda_value, vocab_size):
    emission_probability = {}
    for i in word_tag_unigrams:
        split_tokens = i.split('/')
        word = split_tokens[0]
        tag = split_tokens[-1]
        emission_probability[i] = (word_tag_unigrams[i] + lambda_value) / (tag_unigrams[tag] + (lambda_value * vocab_size))
#         print(word, tag)
#         print(word_tag_unigrams[i], lambda_value)
#         print(tag_unigrams[tag], lambda_value, vocab_size)
#         print(emission_probability[i])
    return emission_probability

In [11]:
print("Get emission probability")
train_emission_probability = get_emission_probability(train_word_tag_unigrams, train_transition_tag_unigrams, 0.1, vocab_size)
# print(train_emission_probability)

Get emission probability


4.4 
Generate 5 random sentences using the previously learned HMM. Output each sentence (with the POS tags) and its probability of being generated.

Hint:
With the help of emission probabilities and transition probabilities collected from 4.2 and 4.3,
    1. Start with ‘<start>’ tag.
    2. Choose next tag based on random choice but considering probabilities e.g. tag draw = random.choices(<tags>,<tag-probabilities>,<numberof tags to draw>).
    3. Now choose word for the corresponding tag using emission probabilities (all the words that can be generated from that tag and corresponding probabilities they can be generated with.)e.g. word draw = random.choices(<words>,<word-probabilities>,<numberof tags to draw>).
    4. Keep repeating steps 2 and 3 till you hit end token ‘</end>.’
    5. Report the sentence and the probability with which this sentence can be generated.

In [12]:
def generate_sentence(transition_probability, emission_probability):
    
    sentence_tags = []
    sentence_words = []
    sentence_transition_probability = []
    sentence_emission_probability = []
    
    while(True):
        # Get potetial next set of tags and its probabilities
        current_tag = "START"
        next_tag = []
        next_tag_probabilities = []
        next_tag_dict = {}
        for i in transition_probability:
            if i.split(" ")[0] == current_tag and i != "END START":
                tag = i.split(" ")[1]
                probability = transition_probability[i]
                next_tag.append(tag)
                next_tag_probabilities.append(probability)
                next_tag_dict[tag] = probability
                
        # Random pick of a tag
        tag_drawn = random.choices(next_tag, next_tag_probabilities)[0]
        
#         print("Tag")
#         print(next_tag)
#         print(next_tag_probabilities)
#         print(tag_drawn)
        
        if (tag_drawn == "END") | (len(sentence_words)==30):
            break
        
        # Get potential words to be chosen out of a selected tag
        next_word = []
        next_word_probabilities = []
        next_word_dict = {}
        for i in emission_probability: 
            if i.split("/")[1] == tag_drawn:
                word = i.split("/")[0]
                probability = emission_probability[i]
                next_word.append(word)
                next_word_probabilities.append(probability)
                next_word_dict[word] = probability
        
        # Random pick of a word
#         print("Word")
#         print(next_word)
#         print(next_word_probabilities)
        word_drawn = random.choices(next_word, next_word_probabilities)[0]

        sentence_tags.append(tag_drawn)
        sentence_words.append(word_drawn)
        sentence_transition_probability.append(next_tag_dict[tag_drawn])
        sentence_emission_probability.append(next_word_dict[word_drawn])
        current_tag = next_tag
        
    total_probability = 1
    for i in range(len(sentence_words)):
        total_probability = total_probability * sentence_transition_probability[i] * sentence_emission_probability[i]
    
    sentence = sentence_words[0]
    for i in range(1, len(sentence_words)):
        sentence = sentence + " " + sentence_words[i]
    return(sentence_tags, sentence_words, sentence_transition_probability, sentence_emission_probability, total_probability, sentence)



In [13]:
sentence_generation_file = open('../output/generated_sentences.txt', 'w')
for i in range(1,6):
    print("Generating sentence: {}".format(i))
    sentence_generation_file.write("Sentence: {}\n".format(1))
    sentence_tags, sentence_words, sentence_transition_probability, \
        sentence_emission_probability, total_probability, sentence = generate_sentence(
            train_transition_probability, train_emission_probability
        )
    sentence_generation_file.write("Words: \n {} \n ".format(sentence_words))
    sentence_generation_file.write("Tags: \n {} \n ".format(sentence_tags))
    sentence_generation_file.write("Transition probability \n {} \n ".format(sentence_transition_probability))
    sentence_generation_file.write("Emission probability \n {} \n ".format(sentence_emission_probability))
    sentence_generation_file.write("Total probability: {} \n \n".format(total_probability))
    sentence_generation_file.write("Sentence: \n {} \n \n".format(sentence))
sentence_generation_file.close()

Generating sentence: 1
Generating sentence: 2
Generating sentence: 3
Generating sentence: 4
Generating sentence: 5


4.5  
For each word in the test dataset, derive the most probable POS tag sequence using the Viterbi algorithm; pseudo-code can be found in the textbook http://web.stanford.edu/ jurafsky/slp3/ed3book.pdf under Figure 8.5. Viterbi algorithm should be implemented following the pseudocode provided for reference.

Hint: Traversing through back-pointer data structure at the end of algorithm would provide information about the best possible previous tag. So when you are at the second last word of the sentence, calling back-pointer here would give the tag information for the first word in the sentence.

Submit the output in a file exactly with the following format (where each line contains no more than one pair):
< sentenceID = 1 >
word/tag
word/tag
....
word/tag
< EOS >
< sentenceID = 2 >
word/tag
word/tag
....
word/tag
< EOS >

In [14]:
def get_state_transition_matrix(states, transition_probability):
    state_transition_df = pd.DataFrame(0, index=states, columns=states)
    for key in transition_probability:
        state1, state2 = key.split(" ")
        state_transition_df.loc[state1, state2] = transition_probability[key]
    return state_transition_df

In [15]:
def get_word_state_emission_matrix(words, states, emission_probability):
    word_state_emission_df = pd.DataFrame(0, index=words, columns=states)
    for key in emission_probability:
        split_tokens = key.split('/')
        word = split_tokens[0]
        tag = split_tokens[-1]
        word_state_emission_df.loc[word, tag] = emission_probability[key]
    return word_state_emission_df

In [16]:
def viterbi_algorithm(words, states, state_transition_df, word_state_emission_df):
    score_df = pd.DataFrame(0, index=range(len(words)), columns=states)
    trace_df = pd.DataFrame("", index=range(len(words)), columns=states)
    for i in range(len(words)):
        # Check for word existence
        word = words[i]
#         print("########## Word: {}".format(words[i]))
        if word not in word_state_emission_df.index:
            print("Converting word: {} to UNK".format(word))
            word = 'UNK'
        
        # For initial word
        if i == 0:
            temp_word_state_emission_list = word_state_emission_df.loc[word, states].values.tolist()
            temp_state_transition_df_list = state_transition_df.loc["START", states].values.tolist()
            temp_score = [temp_word_state_emission_list[i]*temp_state_transition_df_list[i] for i in range(len(temp_word_state_emission_list))]
            score_df.loc[i, states] = temp_score
            trace_df.loc[i, states] = ["START"] * len(states)
#             print(score_df.loc[i, :][score_df.loc[i, :] > 0])
#             print(">>>>>>>>>>>>")
#             print(trace_df.loc[i, :][score_df.loc[i, :] > 0])
#             print(">>>>>>>>>>>>")
        else:
            previous_row_probability_list = score_df.loc[i-1, :].values.tolist()
            for j in range(len(states)):
                current_state = states[j]
                temp_word_state_emission_list = [word_state_emission_df.loc[word, current_state]] * len(states)
                temp_state_transition_df_list = state_transition_df.loc[states, current_state].values.tolist()
                temp_score_list = [
                    previous_row_probability_list[m] * temp_word_state_emission_list[m]*temp_state_transition_df_list[m] 
                    for m in range(len(temp_word_state_emission_list))
                ]
                f = lambda k: temp_score_list[k]
                arg_max_temp_score = max(range(len(temp_score_list)), key=f)
                score_df.loc[i, current_state] = temp_score_list[arg_max_temp_score]
                trace_df.loc[i, current_state] = states[arg_max_temp_score]
#             print(score_df.loc[i, :][score_df.loc[i, :] > 0])
#             print(">>>>>>>>>>>>")
#             print(trace_df.loc[i, :][score_df.loc[i, :] > 0])
#             print(">>>>>>>>>>>>")
            
    # Trace back
    result = []
    # Get the highest score for the last word
    chosen_state = score_df.loc[len(words)-1, :].argmax()
    result.append((words[len(words)-1], chosen_state))
    trace_back_column = trace_df.loc[len(words)-1, chosen_state]
    for i in range(len(words)-2, -1, -1):
        result.append((words[i], trace_back_column))
        trace_back_column = trace_df.loc[i, trace_back_column]
    return result[::-1]    

In [17]:
def get_test_data(file_name):
    file_pointer = open(file_name, "r")
    sentences = []
    sentence = []
    for line in file_pointer:
        
        # Remove duplicate spaces
        file_line_content = re.sub(' +', ' ', line)

        # Strip of begin and end spaces
        file_line_content = file_line_content.strip()

        # If line is not empty
        if file_line_content != "":
            if "sentence ID" in file_line_content:
                continue
            elif  "EOS" in file_line_content:
                sentences.append(sentence)
                sentence = []
            else:
                sentence.append(file_line_content)

    file_pointer.close()
    return sentences

In [18]:
state_transition_df = get_state_transition_matrix(list(train_transition_tag_unigrams.keys()), train_transition_probability)
del state_transition_df['START']

In [20]:
word_state_emission_df = get_word_state_emission_matrix(list(train_word_unigrams.keys()), list(train_transition_tag_unigrams.keys()), train_emission_probability)


In [21]:
test_sentences = get_test_data('../input/Test_File.txt')
viterbi_output_file = open('../output/viterbi_output.txt', 'w')
for i in range(len(test_sentences)):
    print("Processing sentence {}".format(i))
    viterbi_output_file.write("< sentence ID = {} >\n".format(i+1))
#     test_sentences[i] = ['Bella', 'wanted', 'to', 'board', 'the', 'bus', 'to', 'Chicago']
#     test_sentences[i] = ['John', 'nailed', 'the', 'board', 'over', 'the', 'window']
    result = viterbi_algorithm(test_sentences[i],
                               list(train_tag_unigrams.keys()),
                               state_transition_df,
                               word_state_emission_df
                            )
    for word,tag in result:
        viterbi_output_file.write("{}/{}\n".format(word,tag))
    viterbi_output_file.write("< EOS >\n")
viterbi_output_file.close()    

Processing sentence 0
Converting word: kilowatt-hour to UNK
Converting word: kilowatts to UNK
Converting word: kilowatt to UNK
Converting word: $8 to UNK


/Users/karangm/PycharmProjects/pos_tagging/venv/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Processing sentence 1
Converting word: kilowatt-hour to UNK
Processing sentence 2
Processing sentence 3
Converting word: out-of-pocket to UNK
Processing sentence 4
Converting word: electric-utility to UNK
Processing sentence 5
Processing sentence 6
Processing sentence 7
Converting word: utility-cost to UNK
Processing sentence 8
Processing sentence 9
Converting word: subtype to UNK
Converting word: distributes to UNK
Processing sentence 10
Processing sentence 11
Converting word: whereof to UNK
Converting word: hereunto to UNK
Converting word: 11th to UNK
Converting word: sixty-one to UNK
Converting word: eighty-sixth to UNK
Processing sentence 12
Converting word: Resolution to UNK
Converting word: 22nd to UNK
Converting word: Maritime to UNK
Processing sentence 13
Converting word: intermissions to UNK
Processing sentence 14
Processing sentence 15
Converting word: whereof to UNK
Converting word: hereunto to UNK
Converting word: sixty-one to UNK
Converting word: eighty-sixth to UNK
Proces

Converting word: anti-slavery to UNK
Converting word: Finney to UNK
Converting word: revivals to UNK
Processing sentence 149
